[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/scksh/4ch-YOLOv8/blob/main/4ch-YOLOv8.ipynb)

# Install

In [1]:
!git clone https://github.com/scksh/4ch-YOLOv8

Cloning into '4ch-YOLOv8'...
remote: Enumerating objects: 1753, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1753 (delta 13), reused 8 (delta 3), pack-reused 1725 (from 1)
Receiving objects: 100% (1753/1753), 39.14 MiB | 15.10 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [2]:
import os
os.chdir('4ch-YOLOv8')

In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 49.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.34.1
    Uninstalling huggingface-hub-0.34.1:
      Successfully uninstalled huggingface-hub-0.34.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followi

In [4]:
!pip install torch==2.7.0 torchvision==0.22.0 torchaudio==2.7.0 --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.0/867.0 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 9.0 MB/s eta

In [5]:
!pip install -e ../4ch-YOLOv8

Obtaining file:///content/4ch-YOLOv8
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.146-0.editable-py3-none-any.whl size=18546 sha256=f2ef3b9c76f74989acb64bb1d7acfeae5df9e04b92dbabc6f654d66cf566f87b
  Stored in directory: /tmp/pip-ephem-wheel-cache-u4bcnb0o/wheels/10/e5/b2/5d0edfd6a75f4841c2942651e0e3ef7ca11b4ba0269f67685d
Successfully built ultralytics


# Dataset

In [6]:
from huggingface_hub import hf_hub_download
import zipfile

# Download dataset
zip_path = hf_hub_download(
    repo_id="SUMMERZETT/RGBIR",
    filename="RGBIR.zip",
    repo_type="dataset"
)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("./datasets")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RGBIR.zip:   0%|          | 0.00/7.63G [00:00<?, ?B/s]

# Initial Weight

In [7]:
from huggingface_hub import hf_hub_download

# Download pretrained model
model_path = hf_hub_download(
    repo_id="SUMMERZETT/YOLOv8_pretrained",
    filename="yolov8x_4ch_pretrained.pt",
    repo_type="model",
    local_dir="./model",
    local_dir_use_symlinks=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


yolov8x_4ch_pretrained.pt:   0%|          | 0.00/137M [00:00<?, ?B/s]

# Pre-trained model

In [8]:
from huggingface_hub import hf_hub_download

# Download pretrained model
model_path = hf_hub_download(
    repo_id="SUMMERZETT/4ch-YOLOv8",
    filename="4ch-YOLOv8.pt",
    repo_type="model",
    local_dir="./pretrained",
    local_dir_use_symlinks=False
)

4ch-YOLOv8.pt:   0%|          | 0.00/137M [00:00<?, ?B/s]

# Training

In [9]:
!export KMP_DUPLICATE_LIB_OK=TRUE

In [10]:
!yolo task=detect train model=model/yolov8x_4ch_pretrained.pt data=ultralytics/cfg/datasets/RGBIR.yaml workers=2 epochs=1 batch=8 cos_lr=True

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
New https://pypi.org/project/ultralytics/8.3.170 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.146 🚀 Python-3.11.13 torch-2.7.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=ultralytics/cfg/datasets/RGBIR.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=

# Prediction

In [11]:
!python predict.py --weights pretrained/4ch-YOLOv8.pt --source_rgb datasets/RGBIR/images/rgb_val --source_ir datasets/RGBIR/images/thermal --project runs/predict --name 4ch-YOLOv8_pred --imgsz 640 --conf 0.4 --iou 0.5 --device cuda

YOLOv8x summary (fused): 112 layers, 68,126,214 parameters, 0 gradients, 257.5 GFLOPs
Processing images: 100% 221/221 [00:11<00:00, 18.90it/s]

All images visualizations and labels saved: runs/predict/4ch-YOLOv8_pred/save_rgb_vis, runs/predict/4ch-YOLOv8_pred/save_thermal_vis, runs/predict/4ch-YOLOv8_pred/labels
